In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00


In [2]:
%cd /content/drive/MyDrive/AnomalyDetection
!ls

/content/drive/MyDrive/AnomalyDetection
data		   EDA.ipynb   Main.py.ipynb		   output
DataLoading	   loglizer    ModelExperimentation.ipynb  test_metrics.csv
DataPreprocessing  LogParsing  Models			   Utility


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Sep  6 09:26:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from DataPreprocessing.Processing import *
from DataLoading.DataLoader import *
import os
import gc
if __name__ == "__main__":

    print("**************Data Loading and Parameter Setting*****************\n")

    current_directory = os.getcwd()
    data_dir = current_directory + "/data" # Currengt Data Directory
    output_dir = current_directory + "/output" # Output Directory
    log_file = "BGL.log" # Log File
    print(data_dir + "\n" + output_dir)
    dataobj = DataLoader(log_file) # Creating the object of DataLoader Class

    print("**************Return Processed Output Structured and template files after processing Drain Algorithm*****************\n")

    #dataobj.log_parser(data_dir, output_dir, log_file) # Creating structured and template file by using Drain Algorithm


**************Data Loading and Parameter Setting*****************

/content/drive/MyDrive/AnomalyDetection/data
/content/drive/MyDrive/AnomalyDetection/output
**************Return Processed Output Structured and template files after processing Drain Algorithm*****************



# New section

In [5]:
#Setting below functions for the sliding window implementation
window_size = 5
step_size = 1
train_ratio = 0.4
df = pd.read_csv(f'{output_dir}/{log_file}_structured.csv') # Reading Structured file which have addition column EventId and EventTemplate
print(len(df))
print("\n {}".format(df.head()))



4713493

    LineId Label          Id        Date                Code1  \
0       1     -  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   
1       2     -  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   
2       3     -  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   
3       4     -  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   
4       5     -  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   

                         Time                Code2 Component1 Component2  \
0  2005-06-03-15.42.50.363779  R02-M1-N0-C:J12-U11        RAS     KERNEL   
1  2005-06-03-15.42.50.527847  R02-M1-N0-C:J12-U11        RAS     KERNEL   
2  2005-06-03-15.42.50.675872  R02-M1-N0-C:J12-U11        RAS     KERNEL   
3  2005-06-03-15.42.50.823719  R02-M1-N0-C:J12-U11        RAS     KERNEL   
4  2005-06-03-15.42.50.982731  R02-M1-N0-C:J12-U11        RAS     KERNEL   

  Level                                   Content   EventId  \
0  INFO  instruction cache parity error corrected  3aa50e45   
1  INFO  instruction c

In [8]:
    # Data Preprocessing
    print("\n********** Data Pre-Processing***************\n")
    processingobj = Processing(df)
    df = processingobj.process_data(df)
    print("Preprocessed Dataframe \n{}".format(df.head()))
    print("\nCheck Minimum ID Feature Value : \n {}".format(df[['Id']].min()))


********** Data Pre-Processing***************

Preprocessed Dataframe 
   LineId  Label          Id        Date                Code1  \
0       1      0  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   
1       2      0  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   
2       3      0  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   
3       4      0  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   
4       5      0  1117838570  2005.06.03  R02-M1-N0-C:J12-U11   

                         Time                Code2 Component1 Component2  \
0  2005-06-03-15.42.50.363779  R02-M1-N0-C:J12-U11        RAS     KERNEL   
1  2005-06-03-15.42.50.527847  R02-M1-N0-C:J12-U11        RAS     KERNEL   
2  2005-06-03-15.42.50.675872  R02-M1-N0-C:J12-U11        RAS     KERNEL   
3  2005-06-03-15.42.50.823719  R02-M1-N0-C:J12-U11        RAS     KERNEL   
4  2005-06-03-15.42.50.982731  R02-M1-N0-C:J12-U11        RAS     KERNEL   

  Level                                   Content   EventId  \
0  INFO  instruct

In [9]:
    # Sliding Window Implementation

    print("\n Sliding Window implementation\n")
    logdf = dataobj.sliding_windows(df[["timestamp", "Label", "EventId", "deltaT"]],window_size,step_size)
    logdf.head()


 Sliding Window implementation

there are 37315 instances (sliding windows) in this dataset



,timestamp,Label,EventId,deltaT
0,"[1117813370, 1117813370, 1117813370, 111781337...",0,"[3aa50e45, 3aa50e45, 3aa50e45, 3aa50e45, 3aa50...","[0.0, 0.164068, 0.148025, 0.147847, 0.159012, ..."
1,"[1117813430, 1117813430, 1117813430, 111781343...",0,"[3aa50e45, 3aa50e45, 3aa50e45, 3aa50e45, 3aa50...","[0.0, 0.154454, 0.15088, 0.137283, 0.165617, 6..."
2,"[1117813490, 1117813490, 1117813490, 111781349...",0,"[3aa50e45, 3aa50e45, 3aa50e45, 3aa50e45, 3aa50...","[0.0, 0.136003, 0.169535, 0.158248, 0.141224, ..."
3,"[1117813550, 1117813550, 1117813550, 111781355...",0,"[3aa50e45, 3aa50e45, 3aa50e45, 3aa50e45, 3aa50...","[0.0, 0.158966, 0.136016, 0.160414, 0.148952, ..."
4,"[1117813610, 1117813610, 1117813610, 111781361...",0,"[3aa50e45, 3aa50e45, 3aa50e45, 3aa50e45, 3aa50...","[0.0, 0.286636, 0.148804, 0.307791, 0.290853, ..."


In [10]:
    print("Window_size  = 300 , Step_size = 60")

    #Validate Window and stepsize
    print("Validate Windowsize , stepsize implementation min: {},max : {}, window_size : {}".format(np.min(logdf['timestamp'][0]),np.max(logdf['timestamp'][0]),np.max(logdf['timestamp'][0])-np.min(logdf['timestamp'][0])))


Window_size  = 300 , Step_size = 60
Validate Windowsize , stepsize implementation min: 1117813370,max : 1117813669, window_size : 299


In [11]:
    #Segregating Normal and Abnormal data and storing the data to output directory

    print("\nSegregating Normal and Abnormal data and storing the data to output directory\n")

    processingobj.train_test_data(logdf,train_ratio,output_dir)


Segregating Normal and Abnormal data and storing the data to output directory


 Normal Logs Training and Testing

Train normal log size 20578
Test Normal size 13719
Train Abnormal logs size 1810
Test Abnormal logs size 1208

Validate file path
 /content/drive/MyDrive/AnomalyDetection/output/train_abnormal


In [12]:
    #Loading Normal Data

    print("\nLoading Normal Data\n")

    train_normal,test_normal = processingobj.load_normal_data(output_dir)


Loading Normal Data

Train & Test Directory /content/drive/MyDrive/AnomalyDetection/output


/content/drive/MyDrive/AnomalyDetection/DataPreprocessing/Processing.py:84: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_normal = np.array(train_normal).reshape(-1,1)
/content/drive/MyDrive/AnomalyDetection/DataPreprocessing/Processing.py:91: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_normal = np.array(test_normal).reshape(-1,1)


In [13]:
    #Loading Abnormal Data

    print("\nLoading Abnormal Data\n")

    train_abnormal,test_abnormal = processingobj.load_abnormal_data(output_dir)


Loading Abnormal Data

Train & Test Directory /content/drive/MyDrive/AnomalyDetection/output


/content/drive/MyDrive/AnomalyDetection/DataPreprocessing/Processing.py:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_abnormal = np.array(train_abnormal).reshape(-1,1)
/content/drive/MyDrive/AnomalyDetection/DataPreprocessing/Processing.py:113: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_abnormal = np.array(test_abnormal).reshape(-1,1)


In [14]:
    #Consolidate Training normal and abnormal data

    print("\n Consolidate Normal and Abnormal logs - Training set : x_train and y_train\n")

    x_train,y_train = processingobj.consolidate_normal_abnormal_logs(train_normal,train_abnormal,"train")

    assert len(x_train) == len(y_train)

    print("x_train size : {},y_train size :{}".format(len(x_train),len(y_train)))


 Consolidate Normal and Abnormal logs - Training set : x_train and y_train

train Normal and Abnormal dataset
x_train size : 22388,y_train size :22388


In [15]:
type(x_train),type(y_train)

(numpy.ndarray, numpy.ndarray)

In [16]:
x_train[1],y_train[1]

(['3aa50e45', 'ba77ab8e'], 0.0)

In [ ]:
# x_train_transformer = x_train
# y_train_transformer = y_train
# x_train_transformer[1],y_train_transformer[1]

In [17]:
    #Convert EventidTonNumber Training data

    print("Convert EventidTonNumber Training data")

    x_train = processingobj.convert_eventidTonumber(log_file,x_train,output_dir)

Convert EventidTonNumber Training data
Total logkey(exclude 0:UNK) 1325


In [18]:
x_train[1],y_train[1]

([1, 8], 0.0)

In [19]:
    #Consolidate Test normal and abnormal data

    print("\n Consolidate Normal and Abnormal logs - Test set : x_test and y_test\n")

    x_test,y_test = processingobj.consolidate_normal_abnormal_logs(test_normal,test_abnormal,"test")

    assert len(x_test) == len(y_test)

    print("x_test size : {},y_test size :{}".format(len(x_test),len(y_test)))


 Consolidate Normal and Abnormal logs - Test set : x_test and y_test

test Normal and Abnormal dataset
x_test size : 14927,y_test size :14927


In [20]:
type(x_test),type(y_test)

(numpy.ndarray, numpy.ndarray)

In [21]:
x_test[1],y_test[1]

(['1840cbfe', '65f23e3e', '147cfcff', '828a502b'], 0.0)

In [ ]:
# x_test_transformer = x_test
# y_test_transformer = y_test
# x_test_transformer[1],y_test_transformer[1]

In [22]:
    #Convert EventidTonNumber Test data

    print("Convert EventidTonNumber Test data")

    x_test = processingobj.convert_eventidTonumber(log_file,x_test,output_dir)


Convert EventidTonNumber Test data
Total logkey(exclude 0:UNK) 1325


In [23]:
x_test[1],y_test[1]

([4, 6, 7, 5], 0.0)

In [24]:
log_event = pd.read_csv(output_dir + "/" +log_file + "_templates.csv") # read data from template file

eventids = log_event["EventId"].tolist()

print("Total logkey(exclude 0:UNK)", len(eventids)) # log Keys are the evenid's

# 0 means unknown eventids

event_dict = {eid: idx + 1 for idx, eid in enumerate(eventids)}

print(event_dict)

Total logkey(exclude 0:UNK) 1325
{'3aa50e45': 1, 'b9250ad9': 2, 'fc5f0940': 3, '1840cbfe': 4, '828a502b': 5, '65f23e3e': 6, '147cfcff': 7, 'ba77ab8e': 8, '6ede2c6c': 9, '57138c88': 10, '6265c739': 11, 'c3c18d52': 12, '6b9e518d': 13, '6b31e08c': 14, '1e85eb57': 15, '4983ff07': 16, '183ff96a': 17, 'fd2fdc47': 18, 'a31b789f': 19, 'ce2b6cdc': 20, 'a111fe6f': 21, '0b5734c3': 22, '2ade14bf': 23, '1f5adb7d': 24, 'ad738237': 25, '86c84cbc': 26, 'b15df703': 27, '8f4acfdd': 28, '8d23c697': 29, '493f6190': 30, '368b1120': 31, '618cefb8': 32, '1bff92a7': 33, 'f4a80ec4': 34, '3a77b37e': 35, '4aa10e18': 36, '0b246110': 37, '02430fd4': 38, '06072e40': 39, 'c5b54ac5': 40, '0328c31b': 41, 'ab6d0287': 42, '55addd29': 43, 'cf145002': 44, '3777cdd1': 45, '4149b6da': 46, '4029846d': 47, 'cab7c146': 48, '6ce600a9': 49, '12e81061': 50, 'dede275d': 51, 'd4aab023': 52, 'a109aa14': 53, '38869982': 54, '220716fc': 55, 'c70dad25': 56, '22550c73': 57, '53e3af51': 58, 'a5623903': 59, '8fb67bea': 60, '12b2b4b4': 61,

In [25]:
from sklearn.utils import shuffle
import numpy as np
rand_train_index = shuffle(np.arange(len(y_train)), random_state=88)
x_train = x_train[rand_train_index]
y_train = y_train[rand_train_index]

rand_test_index = shuffle(np.arange(len(y_test)), random_state=88)
x_test = x_test[rand_test_index]
y_test = y_test[rand_test_index]

(x_train, y_train), (x_test, y_test)

assert len(x_train) == len(y_train)
assert len(x_test) == len(y_test)

print("Length Validation assertion successful")

Length Validation assertion successful


In [26]:


import argparse
import numpy as np
import pandas as pd
import random
from importlib import reload
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.utils import shuffle
from loglizer import dataloader, preprocessing
from loglizer.utils import metrics
from loglizer.models import  PCA, IsolationForest, OneClassSVM, LogClustering

In [27]:
feature_extractor = preprocessing.FeatureExtractor()
x_train = feature_extractor.fit_transform(x_train)
x_test = feature_extractor.transform(x_test)

====== Transformed train data summary ======
Train data shape: 22388-by-1241

====== Transformed test data summary ======


/content/drive/MyDrive/AnomalyDetection/loglizer/preprocessing.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_df[event] = [0] * len(X_df)
/content/drive/MyDrive/AnomalyDetection/loglizer/preprocessing.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_df[event] = [0] * len(X_df)
/content/drive/MyDrive/AnomalyDetection/loglizer/preprocessing.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all c

Test data shape: 14927-by-1241



/content/drive/MyDrive/AnomalyDetection/loglizer/preprocessing.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_df[event] = [0] * len(X_df)
/content/drive/MyDrive/AnomalyDetection/loglizer/preprocessing.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_df[event] = [0] * len(X_df)
/content/drive/MyDrive/AnomalyDetection/loglizer/preprocessing.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all c

In [28]:
x_train[0],x_train.shape

(array([127.,   0.,   0., ...,   0.,   0.,   0.]), (22388, 1241))

In [29]:
%%time
print("="*20 + " Model: PCA " + "="*20)
for th in np.arange(1):
    print("theshold", th)
    model = PCA(n_components=0.8, threshold=1, c_alpha = 1.9600)
    model.fit(x_train)
    print('Train validation:')
    precision, recall, f1 = model.evaluate(x_train, y_train)
    print('Test validation:')
    precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: PCA ====================
theshold 0
====== Model summary ======
n_components: 5
Project matrix shape: 1241-by-1241
SPE threshold: 1

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 1778, FP: 17881, TN: 2697, FN: 32
Precision: 9.044%, recall: 98.232%, F1-measure: 16.563%

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 1194, FP: 11868, TN: 1851, FN: 14
Precision: 9.141%, recall: 98.841%, F1-measure: 16.734%

CPU times: user 33.7 s, sys: 10.4 s, total: 44.1 s
Wall time: 5.57 s


In [ ]:
%%time
print("="*20 + " Model: IsolationForest " + "="*20)
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=19)
model.fit(x_train)
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: IsolationForest ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 276, FP: 0, TN: 20578, FN: 1534
Precision: 100.000, recall: 15.249, F1-measure: 26.462

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 170, FP: 0, TN: 13719, FN: 1038
Precision: 100.000, recall: 14.073, F1-measure: 24.673

CPU times: user 15.6 s, sys: 8.52 s, total: 24.1 s
Wall time: 24.7 s


In [ ]:
%%time
print("="*20 + " Model: one class SVM " + "="*20)
model = OneClassSVM(kernel='rbf')
model.fit(x_train, y_train)

print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: one class SVM ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 236, FP: 20578, TN: 0, FN: 1574
Precision: 1.134, recall: 13.039, F1-measure: 2.086

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 144, FP: 13719, TN: 0, FN: 1064
Precision: 1.039, recall: 11.921, F1-measure: 1.911

CPU times: user 18min 24s, sys: 1.05 s, total: 18min 25s
Wall time: 18min 26s


In [ ]:
%%time
print("="*20 + " Model: LogClustering " + "="*20)
max_dist = 0.3  # the threshold to stop the clustering process
anomaly_threshold = 0.3  # the threshold for anomaly detection
model = LogClustering(max_dist=max_dist, anomaly_threshold=anomaly_threshold)
model.fit(x_train[y_train == 0, :])  # Use only normal samples for training
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: LogClustering ====================
====== Model summary ======
Starting offline clustering...
Processed 1000 instances.
Found 93 clusters offline.

Starting online clustering...
Processed 2000 instances.
Processed 4000 instances.
Processed 6000 instances.
Processed 8000 instances.
Processed 10000 instances.
Processed 12000 instances.
Processed 14000 instances.
Processed 16000 instances.
Processed 18000 instances.
Processed 20000 instances.
Processed 20578 instances.
Found 180 clusters online.

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 1180, FP: 0, TN: 20578, FN: 630
Precision: 100.000, recall: 65.193, F1-measure: 78.930

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 798, FP: 29, TN: 13690, FN: 410
Precision: 96.493, recall: 66.060, F1-measure: 78.427

CPU times: user 2min 29s, sys: 384 ms, total: 2min 29s
Wall time: 2min 30s


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.6 MB/s eta 0:00:00


In [ ]:
# from sklearn.utils import shuffle
# import numpy as np
# rand_train_index = shuffle(np.arange(len(y_train_transformer)), random_state=88)
# x_train_transformer = x_train_transformer[rand_train_index]
# y_train_transformer = y_train_transformer[rand_train_index]

# rand_test_index = shuffle(np.arange(len(y_test_transformer)), random_state=88)
# x_test_transformer = x_test_transformer[rand_test_index]
# y_test_transformer = y_test_transformer[rand_test_index]

# (x_train_transformer, y_train_transformer), (x_test_transformer, y_test_transformer)

# assert len(x_train_transformer) == len(y_train_transformer)
# assert len(x_test_transformer) == len(y_test_transformer)

# print("Length Validation assertion successful")

Length Validation assertion successful


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
class ClassificationBERTModel(nn.Module):
    def __init__(self, bert_model, num_labels=2):
        super(ClassificationBERTModel, self).__init__()
        self.bert = bert_model
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# Load pre-trained BERT model and tokenizer
bert_model_name = 'bert-base-uncased'
bert_model = BertModel.from_pretrained(bert_model_name)
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [ ]:
# Determine the maximum sequence length
max_seq_length = max(len(seq) for seq in x_train)
max_seq_length

1241

In [ ]:
max_seq_length

1241

In [ ]:
# batch_size = 8
# input_ids_list = []
# attention_mask_list = []

# for sequence in x_train_transformer:
#     # Convert number sequence to string representation and tokenize
#     sequence_str = ' '.join(str(num) for num in sequence)
#     inputs = tokenizer(sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt').to(device)
#     input_ids_list.append(inputs['input_ids'])
#     attention_mask_list.append(inputs['attention_mask'])

In [ ]:
import torch

# Set batch size and max sequence length
batch_size = 16
max_seq_length = 512

# Create empty lists for input data
input_ids_list = []
attention_mask_list = []

# Process sequences in batches
for i in range(0, len(x_train), batch_size):
    batch_sequences = x_train[i:i+batch_size]
    batch_texts = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    # Tokenize batch_texts all at once
    inputs = tokenizer(
        batch_texts,
        padding='max_length',
        truncation=True,
        max_length=max_seq_length,
        return_tensors='pt',
        return_attention_mask=True
    ).to(device)

    input_ids_list.append(inputs['input_ids'])
    attention_mask_list.append(inputs['attention_mask'])

    # Clear GPU memory to manage memory consumption
    if i % 1000 == 0:
        torch.cuda.empty_cache()

In [ ]:
input_ids = torch.cat(input_ids_list, dim=0).to(device)
attention_mask = torch.cat(attention_mask_list, dim=0).to(device)

In [ ]:

# Convert y_train to a tensor
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)  # Move to GPU

# Create DataLoader for training
batch_size = 16
dataset = TensorDataset(input_ids, attention_mask, y_train_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Instantiate the classification model
num_labels = 2
model = ClassificationBERTModel(bert_model, num_labels=num_labels).to('cuda')  # Move to GPU


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        batch = tuple(t.to('cuda') for t in batch)  # Move batch to GPU
        input_ids_batch, attention_mask_batch, y_batch = batch

        optimizer.zero_grad()
        outputs = model(input_ids_batch, attention_mask_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")

Epoch [1/5], Loss: 0.2304
Epoch [2/5], Loss: 0.1301
Epoch [3/5], Loss: 0.0729
Epoch [4/5], Loss: 0.0627
Epoch [5/5], Loss: 0.0694
Training complete!


In [ ]:
# Bert Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'bert_model.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as bert_model.pkl


In [ ]:
# Test

# Evaluation on the test set
model.eval()
test_input_ids_list = []
test_attention_mask_list = []

for batch_start in range(0, len(x_test), batch_size):
    batch_end = batch_start + batch_size
    batch_sequences = x_test[batch_start:batch_end]

    batch_sequence_str = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    batch_inputs = tokenizer.batch_encode_plus(batch_sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt').to(device)

    test_input_ids_list.append(batch_inputs['input_ids'])
    test_attention_mask_list.append(batch_inputs['attention_mask'])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
test_input_ids = torch.cat(test_input_ids_list, dim=0).to('cuda')
test_attention_mask = torch.cat(test_attention_mask_list, dim=0).to('cuda')


In [ ]:
def evaluate_batch(input_ids_batch, attention_mask_batch, y_batch):
    with torch.no_grad():
        test_outputs = model(input_ids_batch, attention_mask_batch)
        _, predicted_labels = torch.max(test_outputs, dim=1)

    predicted_labels = predicted_labels.cpu().numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(y_batch.cpu().numpy(), predicted_labels, average='binary')
    return precision, recall, f1

# Iterate over the test set in batches
test_metrics = []

In [ ]:
batch_size=100
for batch_start in range(0, len(x_test),batch_size):
    batch_end = batch_start + batch_size
    batch_sequences = x_test[batch_start:batch_end]

    batch_sequence_str = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    batch_inputs = tokenizer.batch_encode_plus(batch_sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt')

    input_ids_batch = batch_inputs['input_ids'].to('cuda')
    attention_mask_batch = batch_inputs['attention_mask'].to('cuda')
    y_batch = torch.tensor(y_test[batch_start:batch_end], dtype=torch.long).to('cuda')

    precision, recall, f1 = evaluate_batch(input_ids_batch, attention_mask_batch, y_batch)
    test_metrics.append((precision, recall, f1))

In [ ]:
# Calculate average metrics over all batches
avg_precision = sum(precision for precision, _, _ in test_metrics) / len(test_metrics)
avg_recall = sum(recall for _, recall, _ in test_metrics) / len(test_metrics)
avg_f1 = sum(f1 for _, _, f1 in test_metrics) / len(test_metrics)

print(f"Test Average Precision: {avg_precision:.4f}")
print(f"Test Average Recall: {avg_recall:.4f}")
print(f"Test Average F1-Score: {avg_f1:.4f}")






Test Average Precision: 0.7004
Test Average Recall: 0.6265
Test Average F1-Score: 0.6526


In [ ]:

with torch.no_grad():
    test_outputs = model(test_input_ids, test_attention_mask)
    _, predicted_labels = torch.max(test_outputs, dim=1)

predicted_labels = predicted_labels.cpu().numpy()

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_test, predicted_labels, average='binary')

print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")

In [ ]:
# Tuned Bert Model


In [ ]:
# Create a simple binary classification model using BERT embeddings
class ClassificationBERTModel(nn.Module):
    def __init__(self, bert_model, num_labels=2):
        super(ClassificationBERTModel, self).__init__()
        self.bert = bert_model
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
# Load pre-trained BERT model and tokenizer
bert_model_name = 'bert-base-uncased'
bert_model = BertModel.from_pretrained(bert_model_name)
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [ ]:
# Instantiate the classification model
num_labels = 2
model = ClassificationBERTModel(bert_model, num_labels=num_labels).to('cuda')


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0], device='cuda'))  # Adjust class weights as needed
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 5

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        batch = tuple(t.to('cuda') for t in batch)  # Move batch to GPU
        input_ids_batch, attention_mask_batch, y_batch = batch

        optimizer.zero_grad()
        outputs = model(input_ids_batch, attention_mask_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

Epoch [1/5], Loss: 0.2394
Epoch [2/5], Loss: 0.1118
Epoch [3/5], Loss: 0.1013
Epoch [4/5], Loss: 0.1003
Epoch [5/5], Loss: 0.0945


In [ ]:
# Bert Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'bert_model_tuned.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as bert_model_tuned.pkl


In [ ]:
# Evaluation on the test set
model.eval()

def evaluate_batch(input_ids_batch, attention_mask_batch, y_batch):
    with torch.no_grad():
        test_outputs = model(input_ids_batch, attention_mask_batch)
        _, predicted_labels = torch.max(test_outputs, dim=1)

    predicted_labels = predicted_labels.cpu().numpy()
    accuracy = accuracy_score(y_batch.cpu().numpy(), predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(y_batch.cpu().numpy(), predicted_labels, average='binary')
    return accuracy,precision, recall, f1

In [ ]:
# Iterate over the test set in batches
batch_size=100
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
test_metrics = []
for batch_start in range(0, len(x_test), batch_size):
    batch_end = batch_start + batch_size
    batch_sequences = x_test[batch_start:batch_end]

    batch_sequence_str = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    batch_inputs = tokenizer.batch_encode_plus(batch_sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt')

    input_ids_batch = batch_inputs['input_ids'].to('cuda')
    attention_mask_batch = batch_inputs['attention_mask'].to('cuda')
    y_batch = torch.tensor(y_test[batch_start:batch_end], dtype=torch.long).to('cuda')

    accuracy, precision, recall, f1 = evaluate_batch(input_ids_batch, attention_mask_batch, y_batch)
    test_metrics.append((accuracy, precision, recall, f1))


In [ ]:
# Calculate average metrics over all batches
avg_accuracy = sum(accuracy for accuracy, _, _, _ in test_metrics) / len(test_metrics)
avg_precision = sum(precision for _, precision, _, _ in test_metrics) / len(test_metrics)
avg_recall = sum(recall for _, _, recall, _ in test_metrics) / len(test_metrics)
avg_f1 = sum(f1 for _, _, _, f1 in test_metrics) / len(test_metrics)

print(f"Test Average Accuracy: {avg_accuracy:.4f}")
print(f"Test Average Precision: {avg_precision:.4f}")
print(f"Test Average Recall: {avg_recall:.4f}")
print(f"Test Average F1-Score: {avg_f1:.4f}")

Test Average Accuracy: 0.9715
Test Average Precision: 0.7750
Test Average Recall: 0.9134
Test Average F1-Score: 0.8313


In [ ]:
# DistillBert Algorithm
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:
class ClassificationDistilBERTModel(nn.Module):
    def __init__(self, distilbert_model, num_labels=2):
        super(ClassificationDistilBERTModel, self).__init__()
        self.distilbert = distilbert_model
        self.classifier = nn.Sequential(
            nn.Linear(self.distilbert.config.dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs['last_hidden_state'][:, 0, :]
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
# Load pre-trained DistilBERT model and tokenizer
distilbert_model_name = 'distilbert-base-uncased'
distilbert_model = DistilBertModel.from_pretrained(distilbert_model_name)
tokenizer = DistilBertTokenizer.from_pretrained(distilbert_model_name)

In [ ]:
# Instantiate the classification model
num_labels = 2
model = ClassificationDistilBERTModel(distilbert_model, num_labels=num_labels).to('cuda')


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0], device='cuda'))  # Adjust class weights as needed
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 5

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        batch = tuple(t.to('cuda') for t in batch)  # Move batch to GPU
        input_ids_batch, attention_mask_batch, y_batch = batch

        optimizer.zero_grad()
        outputs = model(input_ids_batch, attention_mask_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")

Epoch [1/5], Loss: 0.2347
Epoch [2/5], Loss: 0.1509
Epoch [3/5], Loss: 0.1399
Epoch [4/5], Loss: 0.1080
Epoch [5/5], Loss: 0.0970
Training complete!


In [ ]:
# DistillBert Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'DistillBert_model_tuned.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as DistillBert_model_tuned.pkl


In [ ]:
# Evaluation on the test set
model.eval()

def evaluate_batch(input_ids_batch, attention_mask_batch, y_batch):
    with torch.no_grad():
        test_outputs = model(input_ids_batch, attention_mask_batch)
        _, predicted_labels = torch.max(test_outputs, dim=1)

    predicted_labels = predicted_labels.cpu().numpy()
    accuracy = accuracy_score(y_batch.cpu().numpy(), predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(y_batch.cpu().numpy(), predicted_labels, average='binary')
    return accuracy,precision, recall, f1

In [ ]:
# Iterate over the test set in batches
batch_size=100
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
test_metrics = []
for batch_start in range(0, len(x_test), batch_size):
    batch_end = batch_start + batch_size
    batch_sequences = x_test[batch_start:batch_end]

    batch_sequence_str = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    batch_inputs = tokenizer.batch_encode_plus(batch_sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt')

    input_ids_batch = batch_inputs['input_ids'].to('cuda')
    attention_mask_batch = batch_inputs['attention_mask'].to('cuda')
    y_batch = torch.tensor(y_test[batch_start:batch_end], dtype=torch.long).to('cuda')

    accuracy, precision, recall, f1 = evaluate_batch(input_ids_batch, attention_mask_batch, y_batch)
    test_metrics.append((accuracy, precision, recall, f1))

In [ ]:
# Calculate average metrics over all batches
avg_accuracy = sum(accuracy for accuracy, _, _, _ in test_metrics) / len(test_metrics)
avg_precision = sum(precision for _, precision, _, _ in test_metrics) / len(test_metrics)
avg_recall = sum(recall for _, _, recall, _ in test_metrics) / len(test_metrics)
avg_f1 = sum(f1 for _, _, _, f1 in test_metrics) / len(test_metrics)

print(f"Test Average Accuracy: {avg_accuracy:.4f}")
print(f"Test Average Precision: {avg_precision:.4f}")
print(f"Test Average Recall: {avg_recall:.4f}")
print(f"Test Average F1-Score: {avg_f1:.4f}")

Test Average Accuracy: 0.9753
Test Average Precision: 0.8315
Test Average Recall: 0.8760
Test Average F1-Score: 0.8452


In [ ]:
#Roberta

In [ ]:
class ClassificationRoBERTaModel(nn.Module):
    def __init__(self, roberta_model, num_labels=2):
        super(ClassificationRoBERTaModel, self).__init__()
        self.roberta = roberta_model
        self.classifier = nn.Sequential(
            nn.Linear(self.roberta.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
# Load pre-trained RoBERTa model and tokenizer
from transformers import RobertaModel, RobertaTokenizer
roberta_model_name = 'roberta-base'
roberta_model = RobertaModel.from_pretrained(roberta_model_name)
tokenizer = RobertaTokenizer.from_pretrained(roberta_model_name)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Instantiate the classification model
num_labels = 2
model = ClassificationRoBERTaModel(roberta_model, num_labels=num_labels).to('cuda')


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0], device='cuda'))  # Adjust class weights as needed
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        batch = tuple(t.to('cuda') for t in batch)  # Move batch to GPU
        input_ids_batch, attention_mask_batch, y_batch = batch

        optimizer.zero_grad()
        outputs = model(input_ids_batch, attention_mask_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")

Epoch [1/5], Loss: 0.3426
Epoch [2/5], Loss: 0.1920
Epoch [3/5], Loss: 0.1560
Epoch [4/5], Loss: 0.1537
Epoch [5/5], Loss: 0.1469
Training complete!


In [ ]:
# Roberta Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'Roberta_model_tuned.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as Roberta_model_tuned.pkl


In [ ]:
# Evaluation on the test set
model.eval()

def evaluate_batch(input_ids_batch, attention_mask_batch, y_batch):
    with torch.no_grad():
        test_outputs = model(input_ids_batch, attention_mask_batch)
        _, predicted_labels = torch.max(test_outputs, dim=1)

    predicted_labels = predicted_labels.cpu().numpy()
    accuracy = accuracy_score(y_batch.cpu().numpy(), predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(y_batch.cpu().numpy(), predicted_labels, average='binary')
    return accuracy,precision, recall, f1

In [ ]:
# Iterate over the test set in batches
batch_size=100
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
test_metrics = []
for batch_start in range(0, len(x_test), batch_size):
    batch_end = batch_start + batch_size
    batch_sequences = x_test[batch_start:batch_end]

    batch_sequence_str = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    batch_inputs = tokenizer.batch_encode_plus(batch_sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt')

    input_ids_batch = batch_inputs['input_ids'].to('cuda')
    attention_mask_batch = batch_inputs['attention_mask'].to('cuda')
    y_batch = torch.tensor(y_test[batch_start:batch_end], dtype=torch.long).to('cuda')

    accuracy, precision, recall, f1 = evaluate_batch(input_ids_batch, attention_mask_batch, y_batch)
    test_metrics.append((accuracy, precision, recall, f1))

In [ ]:
# Calculate average metrics over all batches
avg_accuracy = sum(accuracy for accuracy, _, _, _ in test_metrics) / len(test_metrics)
avg_precision = sum(precision for _, precision, _, _ in test_metrics) / len(test_metrics)
avg_recall = sum(recall for _, _, recall, _ in test_metrics) / len(test_metrics)
avg_f1 = sum(f1 for _, _, _, f1 in test_metrics) / len(test_metrics)

print(f"Test Average Accuracy: {avg_accuracy:.4f}")
print(f"Test Average Precision: {avg_precision:.4f}")
print(f"Test Average Recall: {avg_recall:.4f}")
print(f"Test Average F1-Score: {avg_f1:.4f}")

Test Average Accuracy: 0.0809
Test Average Precision: 0.0809
Test Average Recall: 1.0000
Test Average F1-Score: 0.1485


In [ ]:
# ALberta

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00


In [ ]:
from transformers import AlbertModel, AlbertTokenizer
class ClassificationAlbertModel(nn.Module):
    def __init__(self, albert_model, num_labels=2):
        super(ClassificationAlbertModel, self).__init__()
        self.albert = albert_model
        self.classifier = nn.Sequential(
            nn.Linear(self.albert.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.albert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
# Load pre-trained ALBERT model and tokenizer
albert_model_name = 'albert-base-v2'
albert_model = AlbertModel.from_pretrained(albert_model_name)
tokenizer = AlbertTokenizer.from_pretrained(albert_model_name)

In [ ]:
num_labels = 2
model = ClassificationAlbertModel(albert_model, num_labels=num_labels).to('cuda')


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0], device='cuda'))  # Adjust class weights as needed
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


In [ ]:
# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        batch = tuple(t.to('cuda') for t in batch)  # Move batch to GPU
        input_ids_batch, attention_mask_batch, y_batch = batch

        optimizer.zero_grad()
        outputs = model(input_ids_batch, attention_mask_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")


Epoch [1/5], Loss: 0.4917
Epoch [2/5], Loss: 0.4915
Epoch [3/5], Loss: 0.4495
Epoch [4/5], Loss: 0.3736
Epoch [5/5], Loss: 0.3855
Training complete!


In [ ]:
# ALBERT Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'albert_model_tuned.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as albert_model_tuned.pkl


In [ ]:
# Evaluation on the test set
model.eval()

def evaluate_batch(input_ids_batch, attention_mask_batch, y_batch):
    with torch.no_grad():
        test_outputs = model(input_ids_batch, attention_mask_batch)
        _, predicted_labels = torch.max(test_outputs, dim=1)

    predicted_labels = predicted_labels.cpu().numpy()
    accuracy = accuracy_score(y_batch.cpu().numpy(), predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(y_batch.cpu().numpy(), predicted_labels, average='binary')
    return accuracy,precision, recall, f1

In [ ]:
# Iterate over the test set in batches
batch_size=100
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
test_metrics = []
for batch_start in range(0, len(x_test), batch_size):
    batch_end = batch_start + batch_size
    batch_sequences = x_test[batch_start:batch_end]

    batch_sequence_str = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    batch_inputs = tokenizer.batch_encode_plus(batch_sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt')

    input_ids_batch = batch_inputs['input_ids'].to('cuda')
    attention_mask_batch = batch_inputs['attention_mask'].to('cuda')
    y_batch = torch.tensor(y_test[batch_start:batch_end], dtype=torch.long).to('cuda')

    accuracy, precision, recall, f1 = evaluate_batch(input_ids_batch, attention_mask_batch, y_batch)
    test_metrics.append((accuracy, precision, recall, f1))

In [ ]:
# Calculate average metrics over all batches
avg_accuracy = sum(accuracy for accuracy, _, _, _ in test_metrics) / len(test_metrics)
avg_precision = sum(precision for _, precision, _, _ in test_metrics) / len(test_metrics)
avg_recall = sum(recall for _, _, recall, _ in test_metrics) / len(test_metrics)
avg_f1 = sum(f1 for _, _, _, f1 in test_metrics) / len(test_metrics)

print(f"Test Average Accuracy: {avg_accuracy:.4f}")
print(f"Test Average Precision: {avg_precision:.4f}")
print(f"Test Average Recall: {avg_recall:.4f}")
print(f"Test Average F1-Score: {avg_f1:.4f}")

Test Average Accuracy: 0.8123
Test Average Precision: 0.2077
Test Average Recall: 0.4751
Test Average F1-Score: 0.2803


In [ ]:
#XLNET

In [ ]:
from transformers import XLNetModel, XLNetTokenizer
class ClassificationXLNetModel(nn.Module):
    def __init__(self, xlnet_model, num_labels=2):
        super(ClassificationXLNetModel, self).__init__()
        self.xlnet = xlnet_model
        self.classifier = nn.Sequential(
            nn.Linear(self.xlnet.config.hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.xlnet(input_ids, attention_mask=attention_mask)
        pooled_output = outputs['last_hidden_state'][:, 0]
        logits = self.classifier(pooled_output)
        return logits


In [ ]:

# Load pre-trained XLNet model and tokenizer
xlnet_model_name = 'xlnet-base-cased'
xlnet_model = XLNetModel.from_pretrained(xlnet_model_name)
tokenizer = XLNetTokenizer.from_pretrained(xlnet_model_name)



In [ ]:

# Instantiate the classification model
num_labels = 2
model = ClassificationXLNetModel(xlnet_model, num_labels=num_labels).to('cuda')


In [ ]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0], device='cuda'))  # Adjust class weights as needed
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)



In [ ]:
# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        batch = tuple(t.to('cuda') for t in batch)  # Move batch to GPU
        input_ids_batch, attention_mask_batch, y_batch = batch

        optimizer.zero_grad()
        outputs = model(input_ids_batch, attention_mask_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")


Epoch [1/5], Loss: 0.3798
Epoch [2/5], Loss: 0.2551
Epoch [3/5], Loss: 0.2232
Epoch [4/5], Loss: 0.1892
Epoch [5/5], Loss: 0.1620
Training complete!


In [ ]:
# XLNET Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'xlnet_model_tuned.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as xlnet_model_tuned.pkl


In [ ]:
 #Evaluation on the test set
model.eval()

def evaluate_batch(input_ids_batch, attention_mask_batch, y_batch):
    with torch.no_grad():
        test_outputs = model(input_ids_batch, attention_mask_batch)
        _, predicted_labels = torch.max(test_outputs, dim=1)

    predicted_labels = predicted_labels.cpu().numpy()
    accuracy = accuracy_score(y_batch.cpu().numpy(), predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(y_batch.cpu().numpy(), predicted_labels, average='binary')
    return accuracy,precision, recall, f1

In [ ]:
# Iterate over the test set in batches
batch_size=100
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
test_metrics = []
for batch_start in range(0, len(x_test), batch_size):
    batch_end = batch_start + batch_size
    batch_sequences = x_test[batch_start:batch_end]

    batch_sequence_str = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    batch_inputs = tokenizer.batch_encode_plus(batch_sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt')

    input_ids_batch = batch_inputs['input_ids'].to('cuda')
    attention_mask_batch = batch_inputs['attention_mask'].to('cuda')
    y_batch = torch.tensor(y_test[batch_start:batch_end], dtype=torch.long).to('cuda')

    accuracy, precision, recall, f1 = evaluate_batch(input_ids_batch, attention_mask_batch, y_batch)
    test_metrics.append((accuracy, precision, recall, f1))

In [ ]:
# Calculate average metrics over all batches
avg_accuracy = sum(accuracy for accuracy, _, _, _ in test_metrics) / len(test_metrics)
avg_precision = sum(precision for _, precision, _, _ in test_metrics) / len(test_metrics)
avg_recall = sum(recall for _, _, recall, _ in test_metrics) / len(test_metrics)
avg_f1 = sum(f1 for _, _, _, f1 in test_metrics) / len(test_metrics)

print(f"Test Average Accuracy: {avg_accuracy:.4f}")
print(f"Test Average Precision: {avg_precision:.4f}")
print(f"Test Average Recall: {avg_recall:.4f}")
print(f"Test Average F1-Score: {avg_f1:.4f}")

Test Average Accuracy: 0.0996
Test Average Precision: 0.0822
Test Average Recall: 0.9965
Test Average F1-Score: 0.1506


In [ ]:
#LSTM


In [ ]:
x_train[0],x_train.shape

(array([127.,   0.,   0., ...,   0.,   0.,   0.]), (22388, 1241))

In [ ]:
y_train[0],y_train.shape

(0.0, (22388,))

In [ ]:
np.unique(y_train)

array([0., 1.])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# Define the LSTM-based classification model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use only the final time step output for classification
        return out

In [ ]:
# Hyperparameters
input_size = 1241  # Adjust based on your input data
hidden_size = 256
num_layers = 2
num_classes = 2
batch_size = 32
learning_rate = 0.001
num_epochs = 10

In [ ]:
# Create the LSTM model
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
model.to('cuda')


LSTMModel(
  (lstm): LSTM(1241, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0], device='cuda'))  # Adjust class weights as needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [ ]:
# Convert data to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to('cuda')
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to('cuda')

train_dataset = TensorDataset(x_train_tensor.unsqueeze(1), y_train_tensor)  # Add an extra dimension
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")

Epoch [1/10], Loss: 0.1703
Epoch [2/10], Loss: 0.0764
Epoch [3/10], Loss: 0.0615
Epoch [4/10], Loss: 0.0536
Epoch [5/10], Loss: 0.0500
Epoch [6/10], Loss: 0.0454
Epoch [7/10], Loss: 0.0407
Epoch [8/10], Loss: 0.0409
Epoch [9/10], Loss: 0.0404
Epoch [10/10], Loss: 0.0380
Training complete!


In [ ]:
# LSTM Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'LSTM_model.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as LSTM_model.pkl


In [ ]:
def evaluate(model, dataloader):
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    return accuracy, precision, recall, f1

In [ ]:
# Convert test data to PyTorch tensors
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to('cuda')
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to('cuda')

test_dataset = TensorDataset(x_test_tensor.unsqueeze(1), y_test_tensor)  # Add an extra dimension
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
test_metrics = []
for inputs, labels in test_loader:
    inputs, labels = inputs.to('cuda'), labels.to('cuda')
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)

    test_metrics.append((labels.cpu().numpy(), predicted.cpu().numpy()))

accuracy, precision, recall, f1 = evaluate(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")


Test Accuracy: 0.9898
Test Precision: 0.9363
Test Recall: 0.9371
Test F1-Score: 0.9367


In [ ]:
# Store metrics for each batch for further analysis
all_test_labels, all_test_predictions = zip(*test_metrics)
all_test_labels = np.concatenate(all_test_labels)
all_test_predictions = np.concatenate(all_test_predictions)

overall_precision, overall_recall, overall_f1, _ = precision_recall_fscore_support(all_test_labels, all_test_predictions, average='binary')
overall_accuracy = accuracy_score(all_test_labels, all_test_predictions)

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1-Score: {overall_f1:.4f}")

Overall Accuracy: 0.9898
Overall Precision: 0.9363
Overall Recall: 0.9371
Overall F1-Score: 0.9367


In [ ]:


precision, recall, f1, accuracy = evaluate(model, test_loader)
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Test Precision: 0.9363
Test Recall: 0.9371
Test F1-Score: 0.9367
Test Accuracy: 0.9898


In [ ]:
# Calculate average metrics over all batches
avg_accuracy = sum(accuracy for accuracy, _, _, _ in test_metrics) / len(test_metrics)
avg_precision = sum(precision for _, precision, _, _ in test_metrics) / len(test_metrics)
avg_recall = sum(recall for _, _, recall, _ in test_metrics) / len(test_metrics)
avg_f1 = sum(f1 for _, _, _, f1 in test_metrics) / len(test_metrics)

print(f"Test Average Accuracy: {avg_accuracy:.4f}")
print(f"Test Average Precision: {avg_precision:.4f}")
print(f"Test Average Recall: {avg_recall:.4f}")
print(f"Test Average F1-Score: {avg_f1:.4f}")

In [ ]:
# LSTM Tuned Model

In [ ]:
# Define the LSTM-based classification model
class LSTMTunedModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout):
        super(LSTMTunedModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use only the final time step output for classification
        return out

In [ ]:
# Hyperparameters
input_size = x_train.shape[1]  # Adjust based on your input data
hidden_size = 256
num_layers = 2
num_classes = 2
dropout = 0.2  # Apply dropout for regularization
batch_size = 32
learning_rate = 0.001
num_epochs = 20

In [ ]:
# Create the LSTM model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMTunedModel(input_size, hidden_size, num_layers, num_classes, dropout)
model.to(device)

LSTMTunedModel(
  (lstm): LSTM(1241, 256, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0], device=device))  # Adjust class weights as needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Convert data to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor.unsqueeze(1), y_train_tensor)  # Add an extra dimension
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")

Epoch [1/20], Loss: 0.1778
Epoch [2/20], Loss: 0.0891
Epoch [3/20], Loss: 0.0672
Epoch [4/20], Loss: 0.0604
Epoch [5/20], Loss: 0.0540
Epoch [6/20], Loss: 0.0496
Epoch [7/20], Loss: 0.0456
Epoch [8/20], Loss: 0.0439
Epoch [9/20], Loss: 0.0451
Epoch [10/20], Loss: 0.0429
Epoch [11/20], Loss: 0.0454
Epoch [12/20], Loss: 0.0434
Epoch [13/20], Loss: 0.0371
Epoch [14/20], Loss: 0.0383
Epoch [15/20], Loss: 0.0380
Epoch [16/20], Loss: 0.0377
Epoch [17/20], Loss: 0.0377
Epoch [18/20], Loss: 0.0384
Epoch [19/20], Loss: 0.0389
Epoch [20/20], Loss: 0.0347
Training complete!


In [ ]:
# Tuned LSTM Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'Tuned_LSTM_model.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as Tuned_LSTM_model.pkl


In [ ]:
def evaluate(model, dataloader):
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    return accuracy, precision, recall, f1

In [ ]:
# Convert test data to PyTorch tensors
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to('cuda')
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to('cuda')

test_dataset = TensorDataset(x_test_tensor.unsqueeze(1), y_test_tensor)  # Add an extra dimension
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
test_metrics = []
for inputs, labels in test_loader:
    inputs, labels = inputs.to('cuda'), labels.to('cuda')
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)

    test_metrics.append((labels.cpu().numpy(), predicted.cpu().numpy()))

accuracy, precision, recall, f1 = evaluate(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")


Test Accuracy: 0.9900
Test Precision: 0.9301
Test Recall: 0.9478
Test F1-Score: 0.9389


In [ ]:
# Store metrics for each batch for further analysis
all_test_labels, all_test_predictions = zip(*test_metrics)
all_test_labels = np.concatenate(all_test_labels)
all_test_predictions = np.concatenate(all_test_predictions)

overall_precision, overall_recall, overall_f1, _ = precision_recall_fscore_support(all_test_labels, all_test_predictions, average='binary')
overall_accuracy = accuracy_score(all_test_labels, all_test_predictions)

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1-Score: {overall_f1:.4f}")

Overall Accuracy: 0.9898
Overall Precision: 0.9293
Overall Recall: 0.9462
Overall F1-Score: 0.9377


In [ ]:
# BI-Directional LSTM Model

In [ ]:
# Define the bi-LSTM-based classification model
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout):
        super(BiLSTMModel, self).__init__()
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # Multiply by 2 for bidirectional output

    def forward(self, x):
        out, _ = self.bilstm(x)
        out = self.fc(out[:, -1, :])  # Use only the final time step output for classification
        return out

In [ ]:
# Hyperparameters
input_size = x_train.shape[1]  # Adjust based on your input data
hidden_size = 256
num_layers = 2
num_classes = 2
dropout = 0.2  # Apply dropout for regularization
batch_size = 32
learning_rate = 0.001
num_epochs = 20

In [ ]:
# Create the bi-LSTM model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BiLSTMModel(input_size, hidden_size, num_layers, num_classes, dropout)
model.to(device)

BiLSTMModel(
  (bilstm): LSTM(1241, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=512, out_features=2, bias=True)
)

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0], device=device))  # Adjust class weights as needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Convert data to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor.unsqueeze(1), y_train_tensor)  # Add an extra dimension
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")

Epoch [1/20], Loss: 0.1643
Epoch [2/20], Loss: 0.0841
Epoch [3/20], Loss: 0.0648
Epoch [4/20], Loss: 0.0548
Epoch [5/20], Loss: 0.0494
Epoch [6/20], Loss: 0.0442
Epoch [7/20], Loss: 0.0446
Epoch [8/20], Loss: 0.0430
Epoch [9/20], Loss: 0.0416
Epoch [10/20], Loss: 0.0406
Epoch [11/20], Loss: 0.0399
Epoch [12/20], Loss: 0.0388
Epoch [13/20], Loss: 0.0382
Epoch [14/20], Loss: 0.0384
Epoch [15/20], Loss: 0.0377
Epoch [16/20], Loss: 0.0378
Epoch [17/20], Loss: 0.0353
Epoch [18/20], Loss: 0.0359
Epoch [19/20], Loss: 0.0335
Epoch [20/20], Loss: 0.0337
Training complete!


In [ ]:
# Tuned Bi-LSTM Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'Tuned_Bi-LSTM_model.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as Tuned_Bi-LSTM_model.pkl


In [ ]:
def evaluate(model, dataloader):
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    return accuracy, precision, recall, f1

In [ ]:
# Convert test data to PyTorch tensors
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to('cuda')
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to('cuda')

test_dataset = TensorDataset(x_test_tensor.unsqueeze(1), y_test_tensor)  # Add an extra dimension
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
test_metrics = []
for inputs, labels in test_loader:
    inputs, labels = inputs.to('cuda'), labels.to('cuda')
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)

    test_metrics.append((labels.cpu().numpy(), predicted.cpu().numpy()))

accuracy, precision, recall, f1 = evaluate(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")

Test Accuracy: 0.9917
Test Precision: 0.9601
Test Recall: 0.9363
Test F1-Score: 0.9480


In [ ]:
# Store metrics for each batch for further analysis
all_test_labels, all_test_predictions = zip(*test_metrics)
all_test_labels = np.concatenate(all_test_labels)
all_test_predictions = np.concatenate(all_test_predictions)

overall_precision, overall_recall, overall_f1, _ = precision_recall_fscore_support(all_test_labels, all_test_predictions, average='binary')
overall_accuracy = accuracy_score(all_test_labels, all_test_predictions)

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1-Score: {overall_f1:.4f}")

Overall Accuracy: 0.9928
Overall Precision: 0.9725
Overall Recall: 0.9371
Overall F1-Score: 0.9545


In [ ]:
# Hybrid Model BiLSTM and BERT
!pip install transformers
from transformers import BertTokenizer, BertModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.8 MB/s eta 0:00:00


In [ ]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch.optim as optim

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel, BertTokenizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


In [ ]:
# Define the hybrid model
class HybridModel(nn.Module):
    def __init__(self, bert_model, lstm_input_size, lstm_hidden_size, num_layers, num_classes):
        super(HybridModel, self).__init__()
        self.bert = bert_model
        self.bert_linear = nn.Linear(768, lstm_input_size)  # Linear layer to match LSTM input size
        self.lstm = nn.LSTM(lstm_input_size, lstm_hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(lstm_hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_outputs = self.bert(input_ids, attention_mask=attention_mask)[0]  # Get the hidden states
        bert_hidden_states = self.bert_linear(bert_outputs)
        lstm_out, _ = self.lstm(bert_hidden_states)
        lstm_out = lstm_out[:, -1, :]  # Use only the final time step output for classification
        output = self.fc(lstm_out)
        return output


In [ ]:
# Hyperparameters
bert_model_name = 'bert-base-uncased'
lstm_input_size = 1241  # Adjust based on your input data
lstm_hidden_size = 256
num_lstm_layers = 2
num_classes = 2
batch_size = 32
learning_rate = 0.001
num_epochs = 1

In [ ]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)
bert_model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
# Create the hybrid model
model = HybridModel(bert_model, lstm_input_size, lstm_hidden_size, num_lstm_layers, num_classes)
model.to('cuda')

HybridModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [ ]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.0], device='cuda'))  # Adjust class weights as needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# # Convert data to PyTorch tensors
# x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to('cuda')
# y_train_tensor = torch.tensor(y_train, dtype=torch.long).to('cuda')

# # Tokenize input data and create DataLoader
# train_inputs = tokenizer(x_train, padding=True, truncation=True, return_tensors='pt')
# train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Convert y_train to a tensor
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)  # Move to GPU

# Create DataLoader for training
batch_size = 16
dataset = TensorDataset(input_ids, attention_mask, y_train_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        batch = tuple(t.to('cuda') for t in batch)  # Move batch to GPU
        input_ids_batch, attention_mask_batch, y_batch = batch

        optimizer.zero_grad()
        outputs = model(input_ids_batch, attention_mask_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")

Epoch [1/5], Loss: 0.4862
Epoch [2/5], Loss: 0.4855
Epoch [3/5], Loss: 0.4894
Epoch [4/5], Loss: 0.4902
Epoch [5/5], Loss: 0.4889
Training complete!


In [ ]:
 #Evaluation on the test set
model.eval()

def evaluate_batch(input_ids_batch, attention_mask_batch, y_batch):
    with torch.no_grad():
        test_outputs = model(input_ids_batch, attention_mask_batch)
        _, predicted_labels = torch.max(test_outputs, dim=1)

    predicted_labels = predicted_labels.cpu().numpy()
    accuracy = accuracy_score(y_batch.cpu().numpy(), predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(y_batch.cpu().numpy(), predicted_labels, average='binary')
    return accuracy,precision, recall, f1

In [ ]:
# Iterate over the test set in batches
batch_size=100
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
test_metrics = []
for batch_start in range(0, len(x_test), batch_size):
    batch_end = batch_start + batch_size
    batch_sequences = x_test[batch_start:batch_end]

    batch_sequence_str = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    batch_inputs = tokenizer.batch_encode_plus(batch_sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt')

    input_ids_batch = batch_inputs['input_ids'].to('cuda')
    attention_mask_batch = batch_inputs['attention_mask'].to('cuda')
    y_batch = torch.tensor(y_test[batch_start:batch_end], dtype=torch.long).to('cuda')

    accuracy, precision, recall, f1 = evaluate_batch(input_ids_batch, attention_mask_batch, y_batch)
    test_metrics.append((accuracy, precision, recall, f1))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefin

In [ ]:
# Calculate average metrics over all batches
avg_accuracy = sum(accuracy for accuracy, _, _, _ in test_metrics) / len(test_metrics)
avg_precision = sum(precision for _, precision, _, _ in test_metrics) / len(test_metrics)
avg_recall = sum(recall for _, _, recall, _ in test_metrics) / len(test_metrics)
avg_f1 = sum(f1 for _, _, _, f1 in test_metrics) / len(test_metrics)

print(f"Test Average Accuracy: {avg_accuracy:.4f}")
print(f"Test Average Precision: {avg_precision:.4f}")
print(f"Test Average Recall: {avg_recall:.4f}")
print(f"Test Average F1-Score: {avg_f1:.4f}")

Test Average Accuracy: 0.9191
Test Average Precision: 0.0000
Test Average Recall: 0.0000
Test Average F1-Score: 0.0000


In [ ]:
# Tuned Hybrid Model

In [ ]:
# Load BERT and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class HybridModelTuned(nn.Module):
    def __init__(self, bert_model, lstm_input_size, lstm_hidden_size, num_layers, num_classes, dropout_rate=0.5):
        super(HybridModel, self).__init__()
        self.bert = bert_model
        self.bert_linear = nn.Linear(768, lstm_input_size)
        self.lstm = nn.LSTM(lstm_input_size, lstm_hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(lstm_hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_outputs = self.bert(input_ids, attention_mask=attention_mask)[0]
        bert_hidden_states = self.bert_linear(bert_outputs)
        lstm_out, _ = self.lstm(bert_hidden_states)
        lstm_out = lstm_out[:, -1, :]
        lstm_out = self.dropout(lstm_out)
        output = self.fc(lstm_out)
        return output

In [ ]:
# Hyperparameters
bert_model_name = 'bert-base-uncased'
lstm_input_size = 512  # Adjust based on your input data
lstm_hidden_size = 256
num_lstm_layers = 2
num_classes = 2
num_layers = 2
batch_size = 32
learning_rate = 0.001
num_epochs = 1
dropout_rate = 0.5

In [ ]:
# Initialize the model
model = HybridModel(bert_model, lstm_input_size, lstm_hidden_size, num_layers, num_classes, dropout_rate)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

HybridModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [ ]:
# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        batch = tuple(t.to('cuda') for t in batch)  # Move batch to GPU
        input_ids_batch, attention_mask_batch, y_batch = batch

        optimizer.zero_grad()
        outputs = model(input_ids_batch, attention_mask_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

print("Training complete!")

Epoch [1/10], Loss: 0.2876
Epoch [2/10], Loss: 0.2909
Epoch [3/10], Loss: 0.2867
Epoch [4/10], Loss: 0.2870
Epoch [5/10], Loss: 0.2852
Epoch [6/10], Loss: 0.2867
Epoch [7/10], Loss: 0.2866
Epoch [8/10], Loss: 0.2852
Epoch [9/10], Loss: 0.2858
Epoch [10/10], Loss: 0.2852
Training complete!


In [ ]:
# Tuned Hybrid Model pkl file
import pickle
# Define the path to save the .pkl file
pkl_filename = 'Hybrid_model.pkl'

# Serialize and save the model to a .pkl file
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(model, pkl_file)

print(f"Model saved as {pkl_filename}")

Model saved as Hybrid_model.pkl


In [ ]:
 #Evaluation on the test set
model.eval()

def evaluate_batch(input_ids_batch, attention_mask_batch, y_batch):
    with torch.no_grad():
        test_outputs = model(input_ids_batch, attention_mask_batch)
        _, predicted_labels = torch.max(test_outputs, dim=1)

    predicted_labels = predicted_labels.cpu().numpy()
    accuracy = accuracy_score(y_batch.cpu().numpy(), predicted_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(y_batch.cpu().numpy(), predicted_labels, average='binary')
    return accuracy,precision, recall, f1

In [ ]:
# Iterate over the test set in batches
batch_size=100
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
test_metrics = []
for batch_start in range(0, len(x_test), batch_size):
    batch_end = batch_start + batch_size
    batch_sequences = x_test[batch_start:batch_end]

    batch_sequence_str = [' '.join(str(num) for num in sequence) for sequence in batch_sequences]

    batch_inputs = tokenizer.batch_encode_plus(batch_sequence_str, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt')

    input_ids_batch = batch_inputs['input_ids'].to('cuda')
    attention_mask_batch = batch_inputs['attention_mask'].to('cuda')
    y_batch = torch.tensor(y_test[batch_start:batch_end], dtype=torch.long).to('cuda')

    accuracy, precision, recall, f1 = evaluate_batch(input_ids_batch, attention_mask_batch, y_batch)
    test_metrics.append((accuracy, precision, recall, f1))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: Undefin

In [ ]:
# Calculate average metrics over all batches
avg_accuracy = sum(accuracy for accuracy, _, _, _ in test_metrics) / len(test_metrics)
avg_precision = sum(precision for _, precision, _, _ in test_metrics) / len(test_metrics)
avg_recall = sum(recall for _, _, recall, _ in test_metrics) / len(test_metrics)
avg_f1 = sum(f1 for _, _, _, f1 in test_metrics) / len(test_metrics)

print(f"Test Average Accuracy: {avg_accuracy:.4f}")
print(f"Test Average Precision: {avg_precision:.4f}")
print(f"Test Average Recall: {avg_recall:.4f}")
print(f"Test Average F1-Score: {avg_f1:.4f}")

Test Average Accuracy: 0.9191
Test Average Precision: 0.0000
Test Average Recall: 0.0000
Test Average F1-Score: 0.0000


In [ ]:

gc.collect()

torch.cuda.empty_cache()

%whos


Variable                          Type                Data/Info
---------------------------------------------------------------
BertModel                         type                <class 'transformers.mode<...>modeling_bert.BertModel'>
BertTokenizer                     type                <class 'transformers.mode<...>tion_bert.BertTokenizer'>
ClassificationBERTModel           type                <class '__main__.ClassificationBERTModel'>
DataLoader                        type                <class 'torch.utils.data.dataloader.DataLoader'>
GridSearchCV                      ABCMeta             <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
HybridModel                       type                <class '__main__.HybridModel'>
InvariantsMiner                   type                <class 'loglizer.models.I<...>tsMiner.InvariantsMiner'>
IsolationForest                   ABCMeta             <class 'loglizer.models.I<...>nForest.IsolationForest'>
LR                                typ

# New section